# ai3 for the same thing

## dependencies

In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## dataset stuff

### read the dataset

In [30]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac = 1)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
80,45,1,4,104,208,0,2,148,1,3.0,2,0.0,3.0,0
228,54,1,4,110,206,0,2,108,1,0.0,2,1.0,3.0,3
147,41,1,3,112,250,0,0,179,0,0.0,1,0.0,3.0,0
209,62,0,4,150,244,0,0,154,1,1.4,2,0.0,3.0,1
98,52,1,2,134,201,0,0,158,0,0.8,1,1.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,50,1,4,150,243,0,2,128,0,2.6,2,0.0,7.0,4
42,71,0,2,160,302,0,0,162,0,0.4,1,2.0,3.0,0
137,62,1,2,120,281,0,2,103,0,1.4,2,1.0,7.0,3
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1


### train_test split

In [31]:
ratio = 0.9 # 90:10 train:test ratio
train_set = df.sample(frac=ratio)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = df.drop(train_set.index)
train_set.shape, test_set.shape

train_set
test_set

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
188,54,1,2,192,283,0,2,195,0,0.0,1,1.0,7.0,1
149,60,0,3,102,318,0,0,160,0,0.0,1,1.0,3.0,0
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
250,57,1,4,110,201,0,0,126,1,1.5,2,0.0,6.0,0
157,58,1,4,125,300,0,2,171,0,0.0,1,2.0,7.0,1
186,42,1,3,120,240,1,0,194,0,0.8,3,0.0,7.0,0
19,49,1,2,130,266,0,0,171,0,0.6,1,0.0,3.0,0
75,65,0,3,160,360,0,2,151,0,0.8,1,0.0,3.0,0
139,51,1,3,125,245,1,2,166,0,2.4,2,0.0,3.0,0
162,54,0,3,110,214,0,0,158,0,1.6,2,0.0,3.0,0


### formating

In [37]:
# import training dataset

# Clean column names
train_set.columns = train_set.columns.str.strip()

# Handle missing values represented by '?'
train_set = train_set.replace('?', np.nan)

# Convert columns to numeric, drop rows with NaNs
train_set = train_set.dropna()
train_set = train_set.astype(float)

# Split features and labels
X = train_set.drop("diagnosis", axis=1)
y = train_set["diagnosis"]

# Convert to PyTorch tensors
tensor = torch.tensor(X.values, dtype=torch.float32)
target = torch.tensor(y.values, dtype=torch.float32)  # or LongTensor if it's class labels

print(tensor.shape, target.shape)

torch.Size([267, 13]) torch.Size([267])


In [33]:
# import testing dataset

# Clean column names
test_set.columns = test_set.columns.str.strip()

# Handle missing values represented by '?'
test_set = test_set.replace('?', np.nan)

# Convert columns to numeric, drop rows with NaNs
test_set = test_set.dropna()
test_set = test_set.astype(float)

# Split features and labels
X = test_set.drop("diagnosis", axis=1)
y = test_set["diagnosis"]

# Convert to PyTorch tensors
test_tensor = torch.tensor(X.values, dtype=torch.float32)
test_target = torch.tensor(y.values, dtype=torch.float32)  # or LongTensor if it's class labels

print(test_tensor.shape, test_target.shape)

torch.Size([30, 13]) torch.Size([30])


## ai

### parameters for layers (basically setup)

In [38]:
class SimpleNN(nn.Module): 
    def __init__(self): 
        super(SimpleNN, self).__init__() 
        self.fc1 = nn.Linear(13, 8)  # input layer → hidden layer
        self.relu = nn.ReLU()       # activation
        self.fc2 = nn.Linear(8, 1)  # hidden layer → output layer

    def forward(self, x): 
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # <-- you forgot to call fc3 with (x)
        return x

# Instantiate model
model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=13, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)


### training

In [35]:
#calculate mean squared error
criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr=0.01)

In [36]:
# acc. training
for epoch in range(1000):
    optimiser.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    if torch.isnan(loss):
        print("Loss is NaN!")
        break
    loss.backward()
    optimiser.step()
    
    print(f'Epoch [{epoch + 1}/1000], Loss: {loss.item():.4f}')

NameError: name 'inputs' is not defined

### testing

## ui